<a href="https://colab.research.google.com/github/b-fatma/S2I-DM/blob/master/src/merge/merge_fire_landcover.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import geopandas as gpd
import pandas as pd
import csv
import sys
import geopandas as gpd
import shapely

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
csv.field_size_limit(sys.maxsize)

131072

## Reading fire data

In [ ]:
input_path = '/content/drive/MyDrive/dm_fire_prediction/premerge/fire.csv'

df = pd.read_csv(input_path)
df.head()

,longitude,latitude,fire
0,5.53337,35.70751,1
1,6.46961,32.13579,1
2,6.97630,32.35563,1
3,9.39581,28.19791,1
4,9.49323,28.12826,1


In [ ]:
input_path_lc = '/content/drive/MyDrive/dm_fire_prediction/landcover.csv'
landcover_df = pd.read_csv(input_path_lc)
landcover_df.head()

,geometry,lcccode_label
0,"POLYGON ((6.415278 37.086959, 6.431034 37.0855...",21
1,"POLYGON ((7.180841 37.079167, 7.179981 37.0809...",21
2,"POLYGON ((7.371375 37.081944, 7.370904 37.0871...",21
3,"POLYGON ((6.123611 36.684722, 6.123611 36.6930...",13
4,"POLYGON ((6.261815 37.023611, 6.261927 37.0251...",21


In [ ]:
landcover_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438513 entries, 0 to 438512
Data columns (total 2 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   geometry       438513 non-null  object
 1   lcccode_label  438513 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 6.7+ MB


In [ ]:
gdf_land = landcover_df.copy()
gdf_land["geometry"] = shapely.from_wkt(gdf_land.geometry)
gdf_land = gpd.GeoDataFrame(gdf_land, geometry="geometry", crs="EPSG:4326")

In [ ]:
fire_df = df.copy()

In [ ]:
from shapely.geometry import Point

gdf_fire = gpd.GeoDataFrame(
    fire_df,
    geometry=[Point(xy) for xy in zip(fire_df.longitude, fire_df.latitude)],
    crs="EPSG:4326"
)

In [10]:
result = gpd.sjoin(
    gdf_fire,
    gdf_land[["geometry", "lcccode_label"]],
    how="left",
    predicate="intersects"
)


In [11]:
result = result.drop(columns="index_right")

In [12]:
result

,longitude,latitude,fire,geometry,lcccode_label
0,5.533370,35.707510,1,POINT (5.53337 35.70751),16.0
1,6.469610,32.135790,1,POINT (6.46961 32.13579),19.0
2,6.976300,32.355630,1,POINT (6.9763 32.35563),19.0
3,9.395810,28.197910,1,POINT (9.39581 28.19791),18.0
4,9.493230,28.128260,1,POINT (9.49323 28.12826),18.0
...,...,...,...,...,...
1965251,9.825798,37.320199,0,POINT (9.8258 37.3202),1.0
1965252,9.834781,37.320199,0,POINT (9.83478 37.3202),1.0
1965253,9.843764,37.320199,0,POINT (9.84376 37.3202),1.0
1965254,9.852747,37.320199,0,POINT (9.85275 37.3202),1.0


In [13]:
result.drop(columns='geometry')

,longitude,latitude,fire,lcccode_label
0,5.533370,35.707510,1,16.0
1,6.469610,32.135790,1,19.0
2,6.976300,32.355630,1,19.0
3,9.395810,28.197910,1,18.0
4,9.493230,28.128260,1,18.0
...,...,...,...,...
1965251,9.825798,37.320199,0,1.0
1965252,9.834781,37.320199,0,1.0
1965253,9.843764,37.320199,0,1.0
1965254,9.852747,37.320199,0,1.0


In [16]:
result[result['lcccode_label'].isna()][result['fire'] == 1]

/usr/local/lib/python3.12/dist-packages/geopandas/geodataframe.py:1896: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  result = super().__getitem__(key)


,longitude,latitude,fire,geometry,lcccode_label
3240,3.07880,36.75200,1,POINT (3.0788 36.752),NaN
6496,6.56613,37.01773,1,POINT (6.56613 37.01773),NaN
6523,6.56692,37.02195,1,POINT (6.56692 37.02195),NaN
8773,3.07719,36.75280,1,POINT (3.07719 36.7528),NaN
9292,5.07765,36.74498,1,POINT (5.07765 36.74498),NaN


In [18]:
output_path = '/content/drive/MyDrive/dm_fire_prediction/merge/fire_landcover.csv'
result.to_csv(output_path, index=False)